In [ ]:
import numpy as np
import pyautogui
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time

: 